cGAN Code from https://github.com/asiltureli/gan-in-colab

In [ ]:
!pip install array2gif

In [ ]:
LATENT_SIZE = 100
EPOCH_SIZE  = 200
ROW_NUM     = 10
NUM_CLASSES = 10

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


def plot_epoch(images, n=100):
    '''  
    Visualize a single epoch of images
    Parameters
    ----------
    images : images of shape (b, c, x, y)
    n      : int, default: 100
             number of images to display. Must be a square number
    ''' 
    
    if not isinstance(images, np.ndarray):
        images = images.detach().numpy()

    rowcols = np.sqrt(n) 
    plt.figure(figsize=(rowcols, rowcols))
    for index in range(n):
        plt.subplot(rowcols, rowcols, index + 1)
        plt.imshow(images[index, 0, :, :], cmap="binary")
        plt.axis("off")
    plt.show()

In [ ]:
import torch
import torch.nn as nn

class Generator(nn.Module):
  def __init__(self, latent_size = 100, n_class = NUM_CLASSES):
    super(Generator, self).__init__()
    self.label = nn.Embedding(n_class,10)
    self.layers = nn.Sequential(nn.Linear(latent_size + 10, 512),
                                nn.Dropout(),
                                nn.BatchNorm1d(512, 0.8),
                                nn.LeakyReLU(0.2, inplace=True),
                                nn.Linear(512, 1024),
                                nn.BatchNorm1d(1024, 0.8),
                                nn.LeakyReLU(0.2, inplace=True),
                                nn.Linear(1024,2048),
                                nn.BatchNorm1d(2048, 0.8),
                                nn.LeakyReLU(0.2, inplace=True),
                                nn.Linear(2048,784),
                                nn.Tanh()
                                )       
  def forward(self, latent, labels):
    inp = torch.cat((self.label(labels), latent), 1)
    out = self.layers(inp);
    out = out.view((out.size(0),1,28,28))
    return out

  def __call__(self, latent, labels):
    return self.forward(latent, labels)

class Discriminator(nn.Module):
  def __init__(self, n_class = NUM_CLASSES):
    super(Discriminator, self).__init__()
    self.label = nn.Embedding(n_class, 10)
    self.layers = nn.Sequential(nn.Linear(784 + 10, 1024),                    
                                nn.LeakyReLU(0.2, inplace=True),
                                nn.Linear(1024,512),
                                nn.LeakyReLU(0.2, inplace=True),
                                nn.Linear(512,256),
                                nn.LeakyReLU(0.2, inplace=True),
                                nn.Linear(256, 1),
                                nn.Sigmoid()
                                )
    
  def forward(self, image, labels):
      inp = torch.cat((image.view(image.size(0), -1), self.label(labels)), 1)
      decision = self.layers(inp)
      return decision

  def __call__(self, image, labels):
      return self.forward(image, labels)

In [ ]:
from torch.optim import Adam, SGD
#from torchvision.datasets import MNIST
from array2gif import write_gif
from torch.autograd import Variable
import os
import torchvision

# try:
#   data_path = os.path.join(os.path.abspath(os.environ["CONDA_PREFIX"]),
#                           'datasets')
# except KeyError:
#     data_path = os.path.join(os.path.abspath(os.environ["HOME"]),
#                          'datasets')
# # We make sure that the dataset is actually available
# try:
#     torchvision.datasets.MNIST(root=data_path,
#                                download=False)
# except RuntimeError or KeyError:
#     if not os.path.isdir(data_path):
#             os.makedirs(data_path)
#     torchvision.datasets.MNIST(root=data_path,
#                                download=True)


dataset = torchvision.datasets.MNIST(root=data_path,download=True)
dataset.transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                                    torchvision.transforms.Normalize(mean = (0.5,), 
                                                                                     std = (0.5,))])

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Linear") != -1:
        torch.nn.init.kaiming_uniform_(m.weight.data)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
print("Device is: " + str(device))

# Data Loader
batch_size = 128
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size,
                                          shuffle=True, drop_last=True)
generator     = Generator().to(device).apply(weights_init)
discriminator = Discriminator().to(device).apply(weights_init)

criterion = nn.BCELoss()

optim_d = Adam(discriminator.parameters(), lr=0.00001)
optim_g = Adam(generator.parameters(), lr=0.00002)
gif_array = []*EPOCH_SIZE
total_step = len(data_loader)
for epoch in range(EPOCH_SIZE):
  for step, (imgs, labels) in enumerate(data_loader):

    # Ground truths
    if epoch < 100:
      real_truth = torch.normal(0.9 + (epoch/1000), 0.1001 - (epoch/1000), size = (batch_size, 1)).to(device)
      fake_truth = torch.normal(0.1 - + (epoch/1000), 0.10001 - (epoch/1000), size = (batch_size, 1)).to(device)
    else:
      real_truth = torch.ones(batch_size, 1).to(device)
      fake_truth = torch.zeros(batch_size, 1).to(device)
    # Labels
    real_labels = Variable(labels).to(device)
    real_imgs   = Variable(imgs).to(device)

    # Create latent space and labels
    lat_space = torch.normal(0, 1, size=(batch_size, LATENT_SIZE)).to(device)
    lat_labels = torch.randint(low = 0, high = NUM_CLASSES, size = (batch_size,)).to(device)
    gnrt_imgs = generator(lat_space, lat_labels)

    # ================================================================== #
    #                        Train the generator                         #
    # ================================================================== #
    
    # Compute loss with fake images
    f_outputs = discriminator(gnrt_imgs, lat_labels)
    g_loss = criterion(f_outputs, real_truth)

    # Backprop and optimize
    optim_d.zero_grad()
    optim_g.zero_grad()
    g_loss.backward(retain_graph=True)
    optim_g.step()

    # ================================================================== #
    #                      Train the discriminator                       #
    # ================================================================== #
    gnrt_imgs = generator(lat_space, lat_labels)
    # Real Images
    r_outputs     = discriminator(real_imgs, real_labels)
    d_loss_real = criterion(r_outputs, real_truth)

    # BCE on fake images
    outputs = discriminator(gnrt_imgs, lat_labels)
    d_loss_fake = criterion(outputs, fake_truth)

    # Backprop and optimize
    d_loss = d_loss_real + d_loss_fake
    optim_d.zero_grad()
    optim_g.zero_grad()
    d_loss.backward()
    optim_d.step()

    if (step+1) % 200 == 0:
        print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                .format(epoch, EPOCH_SIZE, 
                        step+1, total_step, 
                        d_loss.item(), 
                        g_loss.item(), 
                        d_loss_real.mean().item(), 
                        d_loss_fake.mean().item()))
  # Generate sample images
  lat_space = torch.randn(NUM_CLASSES**2, LATENT_SIZE).cuda()
  lat_labels = Variable(torch.IntTensor([lab for i in range(NUM_CLASSES) for lab in range(NUM_CLASSES)])).cuda()
  sample_images = generator(lat_space, lat_labels)
  # Plot
  if epoch%5 == 0:
    plot_epoch(sample_images.cpu())
  array_2_make_grid = ((sample_images.cpu()[0:ROW_NUM**2, :, :, :] + 1) * (1/2) * 255).type(torch.uint8)
  gif_array.append(torchvision.utils.make_grid(array_2_make_grid, nrow = ROW_NUM).numpy())
write_gif(gif_array, filename = 'cgan_25_fps.gif', fps = 25)